In [2]:
import cleanlab
import cleanlab_studio
import importlib
from tqdm import tqdm 

importlib.metadata.version('cleanlab')
importlib.metadata.version('cleanlab_studio')

'1.1.18'

In [5]:
import os

studio_api_key = os.environ["STUDIO_API_KEY"]

In [8]:
from cleanlab_studio import Studio

studio = Studio(studio_api_key)
tlm = studio.TLM()

tlm.prompt("<your prompt>")

{'response': 'Write a short story about a young girl who discovers a hidden magical world in her backyard.',
 'confidence_score': 0.4790667746243999}

In [9]:
import pandas as pd

data_path = "/Users/jonas/Dropbox/Cleanlab/AGIhouseHackathon/random500.csv"

chatbot_df = pd.read_csv(data_path)

In [10]:
chatbot_df

,index,question,answer
0,413,Consider the country of Benin. Does the countr...,No
1,2104,Consider the country of Georgia. Does the coun...,Yes
2,6520,Consider the country of Argentina. Does the co...,No
3,6435,Consider the country of Zimbabwe. Does the cou...,No
4,7162,Consider the country of Dominican Republic. Do...,No
...,...,...,...
495,5051,Consider the country of Singapore. Does the co...,No
496,1560,Consider the country of Dominica. Does the cou...,Yes
497,7535,Consider the country of Honduras. Does the cou...,No
498,9035,Consider the country of Turkey. Does the count...,No


In [11]:
tlm = studio.TLM(quality_preset="low")
idx_start = 0  # In case TLM call times-out, can resume loop at this data point

results = chatbot_df.copy(deep=True)  # Store outputs here
results["response"] = None
results["confidence_score"] = None

prompt_template = "Please answer Yes or No only (no punctuation)."

In [12]:
for idx, row in tqdm(results.iloc[idx_start:].iterrows()):
    prompt = prompt_template + " " + row["question"]
    output = tlm.prompt(prompt)
    results.loc[idx, "response"] = output["response"]
    results.loc[idx, "confidence_score"] = output["confidence_score"]

325it [47:14,  8.72s/it]


KeyboardInterrupt: 

In [13]:
results[:10]

,index,question,answer,response,confidence_score
0,413,Consider the country of Benin. Does the countr...,No,No,0.817113
1,2104,Consider the country of Georgia. Does the coun...,Yes,Yes,0.950954
2,6520,Consider the country of Argentina. Does the co...,No,No,0.63109
3,6435,Consider the country of Zimbabwe. Does the cou...,No,No,0.631902
4,7162,Consider the country of Dominican Republic. Do...,No,Yes,0.643741
5,8927,Consider the country of Togo. Does the country...,No,No,0.918669
6,3212,Consider the country of Laos. Does the country...,No,No,0.757725
7,8454,Consider the country of Peru. Does the country...,No,No,0.52404
8,8871,Consider the country of Swaziland. Does the co...,No,No,0.868298
9,6618,Consider the country of Burundi. Does the coun...,No,No,0.813875


In [14]:
results[325:330]

,index,question,answer,response,confidence_score
325,9135,Consider the country of United States of Ameri...,Yes,None,None
326,846,Consider the country of Brunei. Does the count...,No,None,None
327,463,Consider the country of Burkina Faso. Does the...,Yes,None,None
328,617,Consider the country of Bosnia and Herzegovina...,No,None,None
329,3137,Consider the country of South Korea. Does the ...,Yes,None,None


In [15]:
results2 = results[:325]

In [16]:
results2.tail()

,index,question,answer,response,confidence_score
320,6198,Consider the country of Vietnam. Does the coun...,No,No,0.398888
321,8506,Consider the country of Papua New Guinea. Does...,No,No,0.631698
322,4222,Consider the country of Nicaragua. Which resid...,Yes,Yes,0.551833
323,7404,Consider the country of Guinea. Does the count...,Yes,No,0.951028
324,1322,Consider the country of Cape Verde. Does the c...,Yes,No,0.472224


In [17]:
results2["response"].value_counts()

response
No      238
Yes      70
Yes.     17
Name: count, dtype: int64

In [22]:
results2["response"] == 'Yes.'

0      False
1      False
2      False
3      False
4      False
       ...  
320    False
321    False
322    False
323    False
324    False
Name: response, Length: 325, dtype: bool

In [23]:
results2["response"] = results2["response"].replace("Yes.", "Yes")

/var/folders/qy/q1fqr0z967b0szcq6v9dwn2c0000gn/T/ipykernel_49697/1823634509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2["response"] = results2["response"].replace("Yes.", "Yes")


In [24]:
results2["response"].value_counts()

response
No     238
Yes     87
Name: count, dtype: int64

In [25]:
results2.to_csv("tlm_results.csv", index=False)

In [26]:
import numpy as np

(results2["response"] == results2["answer"]).mean()

0.6553846153846153

In [34]:
wrong_responses = results2["response"] != results2["answer"]

In [32]:
results2["confidence_score"] = results2["confidence_score"].astype("float")

/var/folders/qy/q1fqr0z967b0szcq6v9dwn2c0000gn/T/ipykernel_49697/1868764354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2["confidence_score"] = results2["confidence_score"].astype("float")


In [33]:
results2["confidence_score"].describe()

count    325.000000
mean       0.602829
std        0.226961
min        0.000398
25%        0.446727
50%        0.616663
75%        0.786858
max        0.999888
Name: confidence_score, dtype: float64

In [37]:
results2["confidence_score"][wrong_responses].describe()

count    112.000000
mean       0.563410
std        0.224477
min        0.001166
25%        0.414208
50%        0.537807
75%        0.739573
max        0.999888
Name: confidence_score, dtype: float64

In [85]:
threshold = 0.3

low_confidence_responses = results2["confidence_score"] < threshold

low_confidence_responses.sum()

32

In [86]:
(results2["response"][low_confidence_responses] == results2["answer"][low_confidence_responses]).mean()

0.59375

In [70]:
(results2["response"] == results2["answer"]).mean()

0.6553846153846153

In [46]:
results2["confidence_score"].max()

0.9998883044556716

In [88]:
len(results2)

325

In [ ]:
import requests

def google_search(query, api_key, cse_id):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'q': query,
        'key': api_key,
        'cx': cse_id
    }
    response = requests.get(url, params=params)
    return response.json()

# Replace 'your_api_key' and 'your_cse_id' with your actual API key and Custom Search Engine ID
api_key = 'your_api_key'
cse_id = 'your_cse_id'

# Query
query = "how many inches in a foot"

# Perform the search
result = google_search(query, api_key, cse_id)

# Process and print the results
for item in result.get('items', []):
    print(item['title'], item['link'])